In [57]:
import requests
from bs4 import BeautifulSoup
from urllib.request import quote
from selenium import webdriver
import pandas as pd
import time
import re

In [58]:
url = "https://www.hesa.ac.uk/data-and-analysis/students/where-from"

header = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36 Edg/103.0.1264.62"}

html = requests.get(url,header)

#print(html.status_code) #查看是否能获取数据  200

#解析数据
bs1 = BeautifulSoup(html.text, 'html.parser') 
#解析出包含有用信息部分
useful = bs1.find_all(class_="hesa_about_table")

#print(len(useful)) #98条数据

#解析出其中的子链接
repeated_hrefs = []

for u in useful:
    href = u.find('a')['href']
    #print(href)
    repeated_hrefs.append(href)

#子链接去重
de_repeated_hrefs = list(set(repeated_hrefs))
#print(len(de_repeated_hrefs))  #9

print(de_repeated_hrefs)
#['/data-and-analysis/sb262/figure-12', '/data-and-analysis/sb262/figure-11', '/data-and-analysis/students/table-11', '/data-and-analysis/students/table-59', '/data-and-analysis/sb262/figure-9', '/data-and-analysis/students/chart-6', '/data-and-analysis/students/chart-5', '/data-and-analysis/sb262/figure-8', '/data-and-analysis/students/table-28']



['/data-and-analysis/sb262/figure-12', '/data-and-analysis/sb262/figure-11', '/data-and-analysis/students/table-11', '/data-and-analysis/students/table-59', '/data-and-analysis/sb262/figure-9', '/data-and-analysis/students/chart-6', '/data-and-analysis/students/chart-5', '/data-and-analysis/sb262/figure-8', '/data-and-analysis/students/table-28']


In [64]:
res = [] #用于存储info_content字典的列表

sub_url = []  #存储完整子链接
sub_url_pre = "https://www.hesa.ac.uk"

downLinks = [] #存储下载链接

for sub_url_post in de_repeated_hrefs:#遍历每个子页面
    u = sub_url_pre + sub_url_post #组合成完整子链接
    sub_url.append(u)  #存储完整子链接
    
    #爬取每个子页面的数据信息：
    sub_html = requests.get(u, header)
    print(sub_html.status_code) #查看是否能获取数据
    
    #解析子页面数据
    bs2 = BeautifulSoup(sub_html.text, 'html.parser') 
    info = bs2.find(class_="table-metadata")
    
    detail_li = info.find_all("li")
    
    #存储页面特征的字典
    info_content = {}
    #获取前7个属性
    for i in range(7):
        li = detail_li[i].text
        x = li.split(":")
        key = x[0].strip()
        value = x[1].strip()
    
        #print(x[0].strip())
        #print(x[1].strip())    
    
        info_content[key] = value
    
    #获取最近更新时间    
    updateTime = detail_li[len(detail_li)-1].text.split(":")
    info_content[updateTime[0].strip()] = updateTime[1].strip()
    
    #获取下载链接  
    downLink = info.find('a',href = re.compile('.csv'))
    downLinks.append(downLink.text)
    print("下载链接：")
    print(downLink.text)    
    info_content["downLink"] = downLink.text
    
    res.append(info_content)
    print("页面信息：")
    print(info_content)
    

200
下载链接：
https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12.csv
页面信息：
{'Title': 'HE student enrolments based wholly overseas by location of provision, type of provision, level of provision and country of HE provider', 'Reference ID': 'SB262\xa0Figure 12', 'Location': 'UK', 'Academic years': '2016/17\xa0to 2020/21', 'Data source': 'HESA', 'Data collection': '052', 'Sourced from': 'Higher Education Student Statistics', 'Last updated': 'Jan-22', 'downLink': 'https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12.csv'}
200
下载链接：
https://www.hesa.ac.uk/data-and-analysis/sb262/figure-11.csv
页面信息：
{'Title': 'Top ten\xa0EU and non-EU\xa0countries of domicile (excluding the UK) in 2020/21\xa0for HE student enrolments by country of HE provider and first year marker', 'Reference ID': 'SB262 Figure 11', 'Location': 'UK', 'Academic years': '2016/17 to 2020/21', 'Data source': 'HESA', 'Data collection': '051,054', 'Sourced from': 'Higher Education Student Statistics', 'Last updated': 'Jan-2

In [73]:
#列表res转dataFrame
import pandas as pd
info_contents = pd.DataFrame(res)

#存储到excel
#info_contents.to_excel("cross_border_edu_dataForEngland.xlsx")

info_contents.to_csv("cross_border_edu_dataForEngland.csv",index=False,encoding="utf_8_sig")

In [70]:
#总计：爬取了9个页面基本信息 每个页面9个信息

9


In [71]:
#自动下载
print(downLinks)
num = 1
for link in downLinks:
    r = requests.get(link)
    filename = "cross_border_edu_data"+ str(num) + ".csv"
    with open (filename, "wb") as code:
        #code.write(r.content)  写入数据
    num = num + 1
    

['https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12.csv', 'https://www.hesa.ac.uk/data-and-analysis/sb262/figure-11.csv', 'https://www.hesa.ac.uk/data-and-analysis/students/table-11.csv', 'https://www.hesa.ac.uk/data-and-analysis/students/table-59.csv', 'https://www.hesa.ac.uk/data-and-analysis/sb262/figure-9.csv', 'https://www.hesa.ac.uk/data-and-analysis/students/chart-6.csv', 'https://www.hesa.ac.uk/data-and-analysis/students/chart-5.csv', 'https://www.hesa.ac.uk/data-and-analysis/sb262/figure-8.csv', 'https://www.hesa.ac.uk/data-and-analysis/students/table-28.csv']


In [ ]:
sub_url = []
sub_url_pre = "https://www.hesa.ac.uk"

for sub_url_post in de_repeated_hrefs:
    u = sub_url_pre + sub_url_post
    sub_url.append(u)
    
    #爬取每个子页面的数据信息：
    sub_html = request.get(u, header)
    

In [ ]:
'''
urlist = []
drive = webdriver.Chrome()
drive.get(url)
time.sleep(10)
'''

In [9]:
'''
html = drive.page_source
print(html)
'''

<html class="no-js no-touch wf-effra-n4-active wf-effra-n7-active wf-effra-i4-active wf-effra-n5-active wf-effra-n3-active wf-active" lang="en" xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:dc="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:og="http://ogp.me/ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:sioc="http://rdfs.org/sioc/ns#" xmlns:sioct="http://rdfs.org/sioc/types#" xmlns:skos="http://www.w3.org/2004/02/skos/core#" xmlns:xsd="http://www.w3.org/2001/XMLSchema#"><head>
    <title>Where do HE students come from? | HESA</title>

    <meta name="viewport" content="width=device-width, initial-scale=1">
    <meta http-equiv="cleartype" content="on">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <link rel="apple-touch-icon-precomposed" sizes="57x57" href="/sites/all/themes/hesa/favicons/apple-touch-icon-57x57.png">
    <link rel="apple-touch-icon-precomposed" sizes="114x114" href="/sites/all/themes/he

In [11]:
bs1 = BeautifulSoup(html, 'html.parser') #解析数据

useful = bs1.find_all(class_="hesa_about_table")

print(len(useful))

98


In [12]:
repeated_hrefs = []

for u in useful:
    href = u.find('a')['href']
    print(href)
    repeated_hrefs.append(href)



/data-and-analysis/students/table-11
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/data-and-analysis/students/table-59
/

In [13]:
de_repeated_hrefs = list(set(repeated_hrefs))
print(len(de_repeated_hrefs))

9


In [14]:
print(de_repeated_hrefs)

['/data-and-analysis/sb262/figure-12', '/data-and-analysis/sb262/figure-11', '/data-and-analysis/students/table-11', '/data-and-analysis/students/table-59', '/data-and-analysis/sb262/figure-9', '/data-and-analysis/students/chart-6', '/data-and-analysis/students/chart-5', '/data-and-analysis/sb262/figure-8', '/data-and-analysis/students/table-28']


In [17]:
u = "https://www.hesa.ac.uk" + de_repeated_hrefs[0]
sub_html = requests.get(u, header)
print(sub_html.status_code) #查看是否能获取数据


200


In [22]:
bs2 = BeautifulSoup(sub_html.text, 'html.parser') #解析数据
info = bs2.find(class_="table-metadata")
print(info)

<ul class="list-unbulleted table-metadata">
<li><strong>Title</strong>: HE student enrolments based wholly overseas by location of provision, type of provision, level of provision and country of HE provider</li>
<li><strong>Reference ID</strong>: SB262 Figure 12</li>
<li><strong>Location</strong>: UK</li>
<li><strong>Academic years</strong>: 2016/17 to 2020/21</li>
<li><strong>Data source</strong>: HESA</li>
<li><strong>Data collection</strong>: 052</li>
<li><strong>Sourced from</strong>: <a href="https://www.hesa.ac.uk/news/25-01-2022/sb262-higher-education-student-statistics">Higher Education Student Statistics: UK, 2020/21</a></li>
<li><strong>Data source link</strong>: <a href="https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12">https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12</a></li>
<li><strong>Data file canonical link</strong>: <a href="https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12.csv">https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12.csv</a></l

In [ ]:
import pandas as pd
info_contents = pd.DataFrame()

In [23]:
detail_li = info.find_all("li")
print(len(detail_li))

17


In [43]:
info_content = {}
#获取前7个属性
for i in range(7):
    li = detail_li[i].text
    x = li.split(":")
    key = x[0].strip()
    value = x[1].strip()
    
    #print(x[0].strip())
    #print(x[1].strip())    
    
    info_content[key] = value
    
#获取最近更新时间    
updateTime = detail_li[len(detail_li)-1].text.split(":")
info_content[updateTime[0].strip()] = updateTime[1].strip()

print(info_content)

{'Title': 'HE student enrolments based wholly overseas by location of provision, type of provision, level of provision and country of HE provider', 'Reference ID': 'SB262\xa0Figure 12', 'Location': 'UK', 'Academic years': '2016/17\xa0to 2020/21', 'Data source': 'HESA', 'Data collection': '052', 'Sourced from': 'Higher Education Student Statistics', 'Last updated': 'Jan-22'}


In [47]:
#获取下载链接
import re
downLink = info.find('a',href = re.compile('.csv'))
print(downLink.text)

https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12.csv


In [48]:
info_content["downLink"] = downLink.text
print(info_content)

{'Title': 'HE student enrolments based wholly overseas by location of provision, type of provision, level of provision and country of HE provider', 'Reference ID': 'SB262\xa0Figure 12', 'Location': 'UK', 'Academic years': '2016/17\xa0to 2020/21', 'Data source': 'HESA', 'Data collection': '052', 'Sourced from': 'Higher Education Student Statistics', 'Last updated': 'Jan-22', 'downLink': 'https://www.hesa.ac.uk/data-and-analysis/sb262/figure-12.csv'}


In [51]:
print(info_content.keys())

dict_keys(['Title', 'Reference ID', 'Location', 'Academic years', 'Data source', 'Data collection', 'Sourced from', 'Last updated', 'downLink'])
